# Trung Vo NLP Project 
Inspired by this [game](https://en.wikipedia.org/wiki/Hangman_(game) )

In [1]:
import time
import os
import collections
import string

from random import sample 
from nltk.corpus import words

# Model1
Built n-grams model from scratch without NLTK package\
I find that a combination of 1 to 6-grams models is effective, although the number of n-gram models can increase with the extent of computational cost

In [2]:
class Model1(object):
    def __init__(self):        
        self.letter_avail = set(string.ascii_lowercase)
        self.full_dictionary = [word.lower() for word in words.words()]
        self.current_dictionary = self.full_dictionary
        self.ngrams_model(self.current_dictionary)
        
    def ngrams_model(self, current_dictionary):
        self.ngrams = collections.defaultdict(list)
        self.freqs = collections.defaultdict(dict)
        
        for word in current_dictionary:
            for gram in range(1, 7): # gram == number of letter, bigram has gram == 2, we will use 1->6-gram models
                # augment each word with ^ at the beginning and * at the end of the word, number of ^ and * depends on the model
                new_word = "".join(["^"] * (gram-1) + list(word) + ["*"] * (gram-1)) 
                for i in range(len(new_word)):
                    if i+gram > len(new_word):
                        break
                    self.ngrams[gram].append(new_word[i:i+gram])

        for gram in range(1, 7):
            self.freqs[gram] = collections.Counter(self.ngrams[gram])
            
    def guess(self, word):
        guesses = collections.defaultdict(list) # store the "decision" for each letter, we will pick the highest one
        highest_chance = 0
        
        for ch in self.letter_avail:
            # Discounting with Backoff method
            # Run from the highest to the lowest n-grams model until we see the repeated pattern from current dictionary
            for gram in range(6, 1, -1): 
                # augment the word with ^ at the beginning and * at the end of the word to use the corresponding n-grams model
                new_word = "".join(["^"] * (gram-1) + list(word) + ["*"] * (gram-1))
                n = len(new_word)
                skip = False # skip the rest of the loop if we found the pattern in high n-grams model
                for i in range(n):
                    if new_word[i] == '_':
                        # We will consider both cases of the pattern end with [ch] and start with [ch]
                        if i - gram >= 0:
                            tmp = list(new_word[i-gram+1:i])
                            tmp.append(ch)
                            key = "".join(tmp)
                            # replace abcxyz_ with abcxyz[ch] to see if it's in the corresponding n-gram model
                            if key in self.freqs[gram]:
                                guesses[ch].append(self.freqs[gram][key]/self.freqs[gram-1][key[:-1]])
                                # chance of abc[ch] happens = Count(abc[ch])/Count(abc)
                                skip = True

                        if i + gram < n:
                            tmp = [ch]
                            tmp.extend(list(new_word[i+1:i+gram]))
                            key = "".join(tmp)
                            # replace _abcxyz with [ch]abcxyz to see if it's in the corresponding n-gram model
                            if key in self.freqs[gram]:
                                guesses[ch].append(self.freqs[gram][key]/self.freqs[gram-1][key[1:]])
                                skip = True          
                if skip: # found the pattern
                    break
            # Track the ch with the highest chance
            try: 
                ch_chance = sum(guesses[ch])/len(guesses[ch])
                if ch_chance > highest_chance:
                    highest_chance = ch_chance
                    guess_letter = ch
            except:
                pass
        
        # In case of not finding any parttern, usually happens on the first guess
        if highest_chance == 0:
            
            uni_count = collections.Counter()
            for curr in self.current_dictionary:
                # Restrict dictionary for unigram model by len(word)
                if len(curr) != len(word):
                    continue
                # unigram counter is based on number of UNIQUE appearance rather than total appearance in each word
                for ch in set(curr):
                    uni_count[ch] += 1

            most_common_uni = uni_count.most_common()
            i = 0
            while most_common_uni[i][0] not in self.letter_avail: # make sure uni_guess is in letter_avail set
                i += 1
            uni_guess = most_common_uni[i][0]
            return uni_guess
        
        return guess_letter

# Model2

Working in progress, planning to train a Deeplearning model using Transformer.

In [3]:
class Model2(object):
    pass

In [4]:
def play(n_attempts = 6, choice = 1, secret_word = "apple", verbose = True):
    if choice == 1:
        model = Model1()
    elif choice == 2:
        model = Model2()
    attempts_remains = n_attempts
    word = "_"*len(secret_word)
    while attempts_remains > 0:
        if verbose:
            print("# of tries remaining: {0}. Word: {1}.".format(attempts_remains, ' '.join(word)))
        guess_letter = model.guess(word)
        if verbose:
            print('Guessing letter: {0}'.format(guess_letter))
        model.letter_avail.remove(guess_letter)
        if guess_letter in secret_word:
            for i, c in enumerate(secret_word):
                if c == guess_letter:
                    word = list(word)
                    word[i] = c
                    word = ''.join(word)
            if verbose:
                print('Good guess!, updated Word:', ' '.join(word))
        else:
            if verbose:
                print('Wrong guess!, updated Word:', ' '.join(word))
            attempts_remains -= 1
            new_dict = [] # if guess_letter is wrong, we will retrain the model with new_dict
            for voca in model.current_dictionary:
                if set(voca) & set(guess_letter): # new_dict words don't contain wrong guess_letter
                    continue
                new_dict.append(voca)
            model.current_dictionary = new_dict
            model.ngrams_model(model.current_dictionary)

        if '_' not in word:
            if verbose:
                print('Bot won with {0} attempts remains.'.format(attempts_remains))
            return 1
    if not attempts_remains:
        if verbose:
            print('0 attempts left, Bot lost!')
        return 0

# Play here
To manually play, set manual = 1 and change value of the secret_word, lowercase only.

In [5]:
manual = 0
secret_word = "machinelearning"

In [ ]:
if manual:
    play(secret_word = secret_word)
else:
    n_games = 5 # set number of games for AI to play on a test set that is completly different from the train set
    verbose = True # set False if you don't want to see status from the game 
    correct = 0
    secret_words = sample(open('test.txt').read().splitlines(), n_games)
    start = time.time()
    for secret_word in secret_words:
        if verbose:
            print("\nNew game start!")
        correct += play(secret_word = secret_word, verbose = verbose)
    end = time.time()
    print("# games played:", n_games, "# games win:", correct, "accuracy:", correct/n_games)
    print("running time", (end - start)/60, "mins")


New game start!
# of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _.
Guessing letter: e
Good guess!, updated Word: _ _ _ e _ _ _ _ _
# of tries remaining: 6. Word: _ _ _ e _ _ _ _ _.
Guessing letter: r
Good guess!, updated Word: _ _ _ e r _ _ _ _
# of tries remaining: 6. Word: _ _ _ e r _ _ _ _.
Guessing letter: t
Wrong guess!, updated Word: _ _ _ e r _ _ _ _
# of tries remaining: 5. Word: _ _ _ e r _ _ _ _.
Guessing letter: v
Wrong guess!, updated Word: _ _ _ e r _ _ _ _
# of tries remaining: 4. Word: _ _ _ e r _ _ _ _.
Guessing letter: p
Wrong guess!, updated Word: _ _ _ e r _ _ _ _
# of tries remaining: 3. Word: _ _ _ e r _ _ _ _.
Guessing letter: d
Good guess!, updated Word: d _ _ e r _ _ _ _
# of tries remaining: 3. Word: d _ _ e r _ _ _ _.
Guessing letter: i
Good guess!, updated Word: d i _ e r i _ _ _
# of tries remaining: 3. Word: d i _ e r i _ _ _.
Guessing letter: s
Good guess!, updated Word: d i _ e r i s _ s
# of tries remaining: 3. Word: d i _ e r i s _ s.
Guessing letter: u

In [ ]:
try:
    print("List of secret words that the Bot played:")
    print(secret_words)
except:
    pass